- Author: **Võ Bách Khôi** | 19127037
- - -
Notebook này dùng để thu thập dữ liệu từ **Soundcloud**. Chúng ta sẽ thu thập dữ liệu về:
- User
- Playlist
- Track

Đầu tiên là import các thư viện cần thiết.

In [1]:
import requests
import requests_cache
from bs4 import BeautifulSoup
import time
import json
import re
import pandas as pd
import numpy as np
from datetime import datetime

Setup cache trước khi tiến hàng cào dữ liệu :"D

In [2]:
sleep_time = 1
requests_cache.install_cache('http_cache',expire_after=None)

Ta sẽ tiến hành cào dữ liệu của các user đã được chuẩn bị trước.

In [3]:
urls_df = pd.read_csv('urls.csv', index_col=0)
urls_df

,username,url,playlist_sets,playlist
0,lilcandypaint *Candypaint,/lilcandypaint,https://soundcloud.com/lilcandypaint/sets,"/lilcandypaint/sets/its-still-purple,/lilcandy..."
1,Nardo Wick,/nardo-wick,https://soundcloud.com/nardo-wick/sets,NaN
2,Mohamed Ramadan,/mohamedramadanofficial,https://soundcloud.com/mohamedramadanofficial/...,NaN
3,Lil Tjay,/liltjay,https://soundcloud.com/liltjay/sets,/liltjay/sets/no-comparison
4,muhammed fawzii,/muhammed-fawzii,https://soundcloud.com/muhammed-fawzii/sets,"/muhammed-fawzii/sets/music,/muhammed-fawzii/s..."
...,...,...,...,...
2024,MyApple.pl,/myapplepl,https://soundcloud.com/myapplepl/sets,"/myapplepl/sets/myapple-weekly,/myapplepl/sets..."
2025,Tecnocast,/tecnoblog,https://soundcloud.com/tecnoblog/sets,/tecnoblog/sets/tecnoinvest
2026,The Tech Guy,/the_tech_guy,https://soundcloud.com/the_tech_guy/sets,NaN
2027,This Week in Tech,/this-week-in-tech,https://soundcloud.com/this-week-in-tech/sets,NaN


Do **Soundcloud** giới hạn số lần request trong một ngày nên có thể xảy ra lỗi thì chạy hàm này :( Do đó khi có lỗi, ta sẽ trả về những gì đã lấy được

In [4]:
def get_userdata(user_urls_df, user_attributes):
    # Khởi tạo dữ liệu người dùng rỗng
    user_data = []

    for url in user_urls_df:
        # Nếu request xảy ra lỗi, trả về những gì đã lấy được
        try:
            r = requests.get('https://soundcloud.com' + url)
            r.encoding = 'utf-8'
        except:
            return user_data

        # Nếu request không từ cache thì 
        # delay để tránh hit trang web nhiều lần trong thời gian ngắn
        if (r.from_cache == False):
            time.sleep(sleep_time)
        
        # Parse trang web bằng BeautifulSoup
        soup = BeautifulSoup(r.content, 'html.parser')
        
        # Tìm thẻ script chứa "window.__sc_hydration"
        for script in soup.find_all('script'):
            data = script.string
            if data: 
                if 'window.__sc_hydration' in data:
                    data = data[25:-2]
                    idx = data.find('{"hydratable":"user"')
                    if (idx == -1): break
                    data = json.loads(data[idx:])
                    row = [data['data'][att] for att in user_attributes]
                    user_data.append(row)
                    break
        
    return user_data

Chúng ta chỉ lọc ra một số cột quan trọng mà thôi

In [5]:
user_attributes = ['id', 'username', 'verified', 'country_code', 'created_at', 'permalink_url', 
                   'track_count', 'playlist_count', 'followers_count', 'followings_count', 'likes_count']
user_data = get_userdata(urls_df['url'], user_attributes)

In [6]:
user_df = pd.DataFrame(data=user_data, columns=user_attributes)
user_df

,id,username,verified,country_code,created_at,permalink_url,track_count,playlist_count,followers_count,followings_count,likes_count
0,179256009,lilcandypaint *Candypaint,True,US,2015-10-08T03:14:51Z,https://soundcloud.com/lilcandypaint,96,8,44230,13,389
1,811818505,Nardo Wick,False,US,2020-04-18T23:13:35Z,https://soundcloud.com/nardo-wick,11,0,19784,0,6
2,466991778,Mohamed Ramadan,True,EG,2018-06-22T19:31:04Z,https://soundcloud.com/mohamedramadanofficial,51,2,246143,0,0
3,245152552,Lil Tjay,True,None,2016-08-03T12:11:59Z,https://soundcloud.com/liltjay,67,6,784131,2,13
4,76617335,muhammed fawzii,False,EG,2014-01-22T16:11:44Z,https://soundcloud.com/muhammed-fawzii,7,2,2791,28,750
...,...,...,...,...,...,...,...,...,...,...,...
2024,70584473,MyApple.pl,False,PL,2013-12-15T21:15:22Z,https://soundcloud.com/myapplepl,617,4,395,1,1
2025,11458857,Tecnocast,True,BR,2012-01-31T19:41:53Z,https://soundcloud.com/tecnoblog,224,1,7908,3,2
2026,169185192,The Tech Guy,False,None,2015-08-19T02:16:46Z,https://soundcloud.com/the_tech_guy,631,0,534,1,0
2027,169313522,This Week in Tech,False,US,2015-08-19T19:53:56Z,https://soundcloud.com/this-week-in-tech,333,0,862,3,0


In [7]:
user_df.to_csv('user.csv')

Tiếp tục cào dữ liệu các playlist

In [8]:
urls_df['playlist'].fillna('None', inplace=True)

In [9]:
def get_playlistdata(playlist_urls_df, playlist_attributes):
    # Khởi tạo dữ liệu
    playlist_data = []
    
    # Duyệt qua danh sách các playlist của mỗi user
    for playlists in playlist_urls_df:
        # Nếu user này không có playlist nào
        if playlists == 'None':
            pass
        else:
            # Các urls phân cách nhau bởi dấu ','
            playlist_urls = playlists.split(',')
            # Nếu request xảy ra lỗi, trả về những gì đã lấy được
            for playlist_url in playlist_urls:
                try:
                    r = requests.get('https://soundcloud.com' + playlist_url)
                    r.encoding = 'utf8'
                except:
                    return playlist_data
                
                # Nếu request không từ cache thì 
                # delay để tránh hit trang web nhiều lần trong thời gian ngắn
                if (r.from_cache == False):
                    time.sleep(sleep_time)
                
                # Parse trang web bằng BeautifulSoup
                soup = BeautifulSoup(r.content, 'html.parser')
                
                # Tìm thẻ script chứa "window.__sc_hydration"
                for script in soup.find_all('script'):
                    data = script.string
                    if data: 
                        if 'window.__sc_hydration' in data:
                            data = data[25:-2]
                            idx = data.find('{"hydratable":"playlist"')
                            if (idx == -1): break
                            data = json.loads(data[idx:])
                            row = [data['data'][att] for att in playlist_attributes]
                            playlist_data.append(row)
                            break
                            
    return playlist_data

In [10]:
playlist_attributes = ['created_at', 'id', 'likes_count', 'permalink_url', 'reposts_count', 'genre', 'license', 'title', 'user_id', 'tracks']
playlist_data = get_playlistdata(urls_df['playlist'], playlist_attributes)

In [11]:
playlist_df = pd.DataFrame(data=playlist_data, columns=playlist_attributes)
playlist_df

,created_at,id,likes_count,permalink_url,reposts_count,genre,license,title,user_id,tracks
0,2020-02-18T23:56:36Z,994786066,2331,https://soundcloud.com/lilcandypaint/sets/its-...,150,,all-rights-reserved,its still purple,179256009,[{'artwork_url': 'https://i1.sndcdn.com/artwor...
1,2018-11-05T23:23:03Z,638014500,1769,https://soundcloud.com/lilcandypaint/sets/s8ta...,147,,all-rights-reserved,S8TANS LAST DANCE,179256009,[{'artwork_url': 'https://i1.sndcdn.com/artwor...
2,2018-11-05T23:21:56Z,638013903,1078,https://soundcloud.com/lilcandypaint/sets/s8ta...,83,,all-rights-reserved,S8TANS LAST DANCE,179256009,[{'artwork_url': 'https://i1.sndcdn.com/artwor...
3,2016-08-30T01:31:01Z,254786558,460,https://soundcloud.com/lilcandypaint/sets/inte...,52,INTERNETMUSIC,all-rights-reserved,INTERNET,179256009,[{'artwork_url': 'https://i1.sndcdn.com/artwor...
4,2018-12-24T22:00:08Z,670946511,21484,https://soundcloud.com/liltjay/sets/no-comparison,1062,,all-rights-reserved,No Comparison,245152552,[{'artwork_url': 'https://i1.sndcdn.com/artwor...
...,...,...,...,...,...,...,...,...,...,...
3760,2017-10-14T19:00:50Z,359984842,1,https://soundcloud.com/myapplepl/sets/myapple-...,0,,all-rights-reserved,MyApple Weekly,70584473,[{'artwork_url': 'https://i1.sndcdn.com/artwor...
3761,2014-12-09T22:39:17Z,63234214,43,https://soundcloud.com/myapplepl/sets/myapple-...,2,Technology,all-rights-reserved,MyApple Daily,70584473,"[{'artwork_url': None, 'caption': None, 'comme..."
3762,2013-12-22T20:16:03Z,17908634,40,https://soundcloud.com/myapplepl/sets/macgadka,3,Technology,all-rights-reserved,MacGadka 🎙,70584473,"[{'artwork_url': None, 'caption': None, 'comme..."
3763,2013-12-16T10:16:59Z,17326482,6,https://soundcloud.com/myapplepl/sets/podcast-...,1,,all-rights-reserved,Podcast MyApple 2006 Season 01,70584473,"[{'artwork_url': None, 'caption': None, 'comme..."


In [12]:
track_raw_data = playlist_df['tracks'].copy()

In [13]:
track_ids = []
for tracks in playlist_df['tracks']:
    row = [str(track['id']) for track in tracks]
    track_ids.append(','.join(row))
playlist_df['tracks'] = track_ids

In [14]:
playlist_df

,created_at,id,likes_count,permalink_url,reposts_count,genre,license,title,user_id,tracks
0,2020-02-18T23:56:36Z,994786066,2331,https://soundcloud.com/lilcandypaint/sets/its-...,150,,all-rights-reserved,its still purple,179256009,"762943381,762943480,762943408,762943528"
1,2018-11-05T23:23:03Z,638014500,1769,https://soundcloud.com/lilcandypaint/sets/s8ta...,147,,all-rights-reserved,S8TANS LAST DANCE,179256009,"525364131,525364092,525364122,525364077"
2,2018-11-05T23:21:56Z,638013903,1078,https://soundcloud.com/lilcandypaint/sets/s8ta...,83,,all-rights-reserved,S8TANS LAST DANCE,179256009,"525364092,525364131,525364122,525364077"
3,2016-08-30T01:31:01Z,254786558,460,https://soundcloud.com/lilcandypaint/sets/inte...,52,INTERNETMUSIC,all-rights-reserved,INTERNET,179256009,280512672
4,2018-12-24T22:00:08Z,670946511,21484,https://soundcloud.com/liltjay/sets/no-comparison,1062,,all-rights-reserved,No Comparison,245152552,"549660528,549660318,549659919,549659742,549658101"
...,...,...,...,...,...,...,...,...,...,...
3760,2017-10-14T19:00:50Z,359984842,1,https://soundcloud.com/myapplepl/sets/myapple-...,0,,all-rights-reserved,MyApple Weekly,70584473,"344973297,345914202,346903456,347914874,349169..."
3761,2014-12-09T22:39:17Z,63234214,43,https://soundcloud.com/myapplepl/sets/myapple-...,2,Technology,all-rights-reserved,MyApple Daily,70584473,"180772908,180913275,181043946,181207702,181649..."
3762,2013-12-22T20:16:03Z,17908634,40,https://soundcloud.com/myapplepl/sets/macgadka,3,Technology,all-rights-reserved,MacGadka 🎙,70584473,"126061195,128131427,130886926,132869020,133435..."
3763,2013-12-16T10:16:59Z,17326482,6,https://soundcloud.com/myapplepl/sets/podcast-...,1,,all-rights-reserved,Podcast MyApple 2006 Season 01,70584473,"125005362,125003313,126066543,126069672,126071..."


In [15]:
playlist_df.to_csv('playlist.csv')

Từ các playlist đã thu thập, chọn ra tối đa 5 tracks đầu tiên để lấy dữ liệu

In [16]:
track_raw_data

0       [{'artwork_url': 'https://i1.sndcdn.com/artwor...
1       [{'artwork_url': 'https://i1.sndcdn.com/artwor...
2       [{'artwork_url': 'https://i1.sndcdn.com/artwor...
3       [{'artwork_url': 'https://i1.sndcdn.com/artwor...
4       [{'artwork_url': 'https://i1.sndcdn.com/artwor...
                              ...                        
3760    [{'artwork_url': 'https://i1.sndcdn.com/artwor...
3761    [{'artwork_url': None, 'caption': None, 'comme...
3762    [{'artwork_url': None, 'caption': None, 'comme...
3763    [{'artwork_url': None, 'caption': None, 'comme...
3764    [{'artwork_url': 'https://i1.sndcdn.com/artwor...
Name: tracks, Length: 3765, dtype: object

In [17]:
track_attribute = ['created_at', 'id', 'likes_count', 'permalink_url', 'reposts_count', 'comment_count', 'download_count', 
                   'downloadable', 'duration', 'full_duration', 'genre', 'license', 'title', 'user_id']
track_data = []

for tracks in track_raw_data:
    for track in tracks:
        if 'created_at' in track:
            row = [track[att] for att in track_attribute]
            track_data.append(row)

track_df = pd.DataFrame(data=track_data, columns=track_attribute)
track_df

,created_at,id,likes_count,permalink_url,reposts_count,comment_count,download_count,downloadable,duration,full_duration,genre,license,title,user_id
0,2020-02-18T23:55:03Z,762943381,3584.0,https://soundcloud.com/lilcandypaint/selfishpr...,125,84.0,0.0,False,120137,120137,Hip-hop & Rap,all-rights-reserved,Selfish(prod.menohbeats),179256009
1,2020-02-18T23:55:22Z,762943480,3545.0,https://soundcloud.com/lilcandypaint/angles-pr...,110,61.0,0.0,False,133016,133016,Hip-hop & Rap,all-rights-reserved,Angles (Prod. 5heriff X Hippiedren),179256009
2,2020-02-18T23:55:12Z,762943408,3655.0,https://soundcloud.com/lilcandypaint/fishtail-...,128,106.0,0.0,False,115801,115801,Hip-hop & Rap,all-rights-reserved,fishtail (Prod.5heriff),179256009
3,2020-02-18T23:55:30Z,762943528,3150.0,https://soundcloud.com/lilcandypaint/2018-prod...,121,100.0,0.0,False,104072,104072,Hip-hop & Rap,all-rights-reserved,2018 (prod.litooo),179256009
4,2018-11-05T23:18:05Z,525364131,2577.0,https://soundcloud.com/lilcandypaint/lilcandyp...,102,37.0,0.0,False,83520,83520,Hip-hop & Rap,all-rights-reserved,lilcandypaint- clouds (prod. Trapboytango) [HO...,179256009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14182,2016-06-03T00:19:48Z,267194823,NaN,https://soundcloud.com/tecnoblog/042-independe...,0,NaN,NaN,True,6455645,6455645,Tecnologia,cc-by-nc-sa,042 – Independência Financeira,11458857
14183,2016-10-07T00:10:43Z,286472803,NaN,https://soundcloud.com/tecnoblog/050-investind...,0,NaN,NaN,True,4981077,4981077,Negócio,cc-by-nc-sa,050 – Investindo com Robôs,11458857
14184,2017-08-11T00:28:14Z,337373406,NaN,https://soundcloud.com/tecnoblog/071-vale-a-pe...,0,NaN,NaN,True,4799012,4799012,Negócios,cc-by-nc-sa,071 - Vale a pena investir em bitcoin?,11458857
14185,2017-09-21T22:34:33Z,343455746,NaN,https://soundcloud.com/tecnoblog/074-investind...,0,NaN,NaN,True,5180985,5180985,Negócios,cc-by-nc-sa,074 – Investindo com Robôs pt2,11458857


In [18]:
track_df.to_csv('track.csv')